# 06 Integración de Sistemas (ERP y SCM)

En este ejercicio vamos a programar la lógica de negocio que permite a un sistema ERP integrar ventas con inventario y a un sistema SCM automatizar pedidos a proveedores.

**Instrucciones:**
1. Leer el archivo de base de datos (`inventario_ERPSCM.csv`).
2. Completa las funciones en cada una de las celdas de las Partes 1, 2, y 3.
3. Ejecuta la celda de la **Parte 4** para probar si las funciones están correctas.
4. Repetir el paso 3. con más 3 productos diferentes.

In [16]:
import pandas as pd

In [17]:
# Usamos Pandas para una lectura limpia de los datos
base_datos_erp = pd.read_csv(r'C:\Users\P3E010-D\Downloads\inventario_ERPSCM.csv')
# Convertimos a lista de diccionarios para facilitar la iteración en este ejercicio

# Inicializamos la "Base de Datos" en memoria
ventas_totales_dia = 0.0

print(f"✅ Datos cargados: {len(base_datos_erp)} productos en el sistema ERP.")
# Mostramos los datos usando Pandas para que se vea bonita la tabla
display(pd.DataFrame(base_datos_erp))

✅ Datos cargados: 302 productos en el sistema ERP.


,id,nombre,precio,stock_actual,punto_reorden,pronostico_venta
0,100,Laptop Gamer Muestra (Bajo Stock),1500.00,2,10,25
1,101,Cable HDMI Básico (Stock Alto),15.00,200,20,50
2,102,Audífonos Noise Cancelling Lenovo Gamer,306.71,79,26,88
3,103,Mouse Inalámbrico Xiaomi Slim,90.75,83,23,86
4,104,Lavadora Apple Business,585.53,17,26,23
...,...,...,...,...,...,...
297,397,Smartphone Lenovo Pro,1325.42,79,22,80
298,398,Parlante Bluetooth Xiaomi Pro,139.87,67,30,71
299,399,Laptop HP Max,1927.56,71,26,68
300,400,Laptop LG Pro,1284.85,41,18,48


## Parte 1: Módulo ERP (Ventas e Inventario)

Verifica si hay suficiente stock

In [18]:
def procesar_venta(id_producto, cantidad_deseada):
    global ventas_totales_dia
    producto_encontrado = False

    print(f"Se desea vender {cantidad_deseada} unidad(es) del producto {id_producto}...")

    for producto in base_datos_erp:
        if producto['id'] == id_producto:
            producto_encontrado = True
            
            # --- TAREA: Lógica ERP ---
            # Verifica si hay suficiente stock
            if producto['stock_actual'] >= cantidad_deseada: # <--- COMPLETA AQUÍ
                
                # Actualiza el stock
                producto['stock_actual'] = producto['stock_actual'] - cantidad_deseada # <--- COMPLETA AQUÍ
                
                # Calcula el ingreso (Precio * Cantidad)
                ingreso = producto['precio'] * cantidad_deseada # <--- COMPLETA AQUÍ
                ventas_totales_dia += ingreso
                
                print(f"   Transacción exitosa: {producto['nombre']}. Nuevo stock: {producto['stock_actual']}")
            else:
                print(f"   Error: Stock insuficiente para {producto['nombre']}.")
    
    if not producto_encontrado:
        print("   Error: Producto no encontrado.")

print("Módulo ERP cargado.")

Módulo ERP cargado.


## Parte 2: Módulo SCM - Modelo PUSH

Si el stock actual es menor al pronóstico, debemos comprar la diferencia

In [19]:
def ejecutar_modelo_push():
    print("\nEJECUTANDO MODELO SCM PUSH (MAKE-TO-STOCK)")
    print("Regla: Comprar anticipadamente si Stock Actual < Pronóstico.")
    
    compras_realizadas = 0
    
    for producto in base_datos_erp:
        stock = producto['stock_actual']
        pronostico = producto['pronostico_venta']
        
        # --- TAREA: Lógica Push ---
        # Si el stock actual es menor al pronóstico, debemos comprar la diferencia
        if stock < pronostico:
            
            # Calcula cuánto falta para alcanzar el pronóstico
            cantidad_a_comprar = pronostico- stock  # <--- COMPLETA AQUÍ
            
            print(f"   PUSH: {producto['nombre']} (Stock: {stock} | Pronóstico: {pronostico})")
            print(f"   Generando orden de compra por {cantidad_a_comprar} unidades.")
            
            # Simulamos llegada de mercadería
            producto['stock_actual'] += cantidad_a_comprar
            compras_realizadas += 1
            
    if compras_realizadas == 0:
        print("   PUSH Reporte: Inventario suficiente para los pronósticos.")

print("Módulo SCM (Push) cargado.")

Módulo SCM (Push) cargado.


## Parte 3: Módulo SCM - Modelo PULL

Si no hay stock, el modelo Pull dispara la compra

In [20]:
def ejecutar_modelo_pull(id_producto, cantidad_cliente):
    print(f"\n🚚 EJECUTANDO MODELO PULL (MAKE-TO-ORDER)")
    print(f"Cliente solicita {cantidad_cliente} unidades del producto {id_producto}...")
    
    for producto in base_datos_erp:
        if producto['id'] == id_producto:
            stock = producto['stock_actual']
            
            # --- TAREA 3: Lógica Pull ---
            if stock >= cantidad_cliente:
                print(f"   ✅ Stock suficiente ({stock}). Se despacha del inventario existente.")
                producto['stock_actual'] -= cantidad_cliente
            else:
                # Si no hay stock, el modelo Pull dispara la compra JUSTO AHORA
                faltante = cantidad_cliente - stock # <--- COMPLETA AQUÍ
                
                print(f"   Stock insuficiente. Faltan {faltante} unidades.")
                print(f"   -> PULL ACTIVADO: Comprando {faltante} unidades urgentemente...")
                
                print(f"   ✅ Pedido entregado al cliente (Just-in-Time).")
                # En Pull puro, el stock queda en 0 tras la entrega inmediata (si no se pidió extra)
                producto['stock_actual'] = 0 

print("Módulo SCM (Pull) cargado.")

Módulo SCM (Pull) cargado.


## Parte 4: Integración ERP - SCM

Si no hay stock, el modelo Pull dispara la compra

In [22]:
# 1. Reiniciamos la base de datos para asegurar una prueba limpia
ventas_totales_dia = 0.0

print("--- 1. PROBANDO ERP (VENTAS) ---")
procesar_venta(101, 1)  # ¿Funciona o falla?
procesar_venta(103, 10) # ¿Funciona o falla?

# Se debe crear un caso en el que la venta funcione,
# y otro caso en el que la venta falle

print(f"\nDinero en Caja (Finanzas): ${ventas_totales_dia}")

print("\n--- 2. PROBANDO SCM (PUSH) ---")
# Esto debería reabastecer un producto que no se haya podido vender
ejecutar_modelo_push()

print("\n--- 3. PROBANDO SCM (PULL) ---")
# Se debe pedir un producto en una cantidad superior al stock
# El sistema Pull debería comprar los faltantes automáticamente
ejecutar_modelo_pull(105, 200)

print("\n📊 ESTADO FINAL DEL INVENTARIO (ERP INTEGRADO)")
# Convertimos de nuevo a DataFrame para ver la tabla bonita en Jupyter
display(pd.DataFrame(base_datos_erp))

--- 1. PROBANDO ERP (VENTAS) ---
Se desea vender 1 unidad(es) del producto 101...


TypeError: string indices must be integers, not 'str'